### CICModbus2023 Flow labeling 

In [1]:
import os
import pandas as pd
import numpy as np
import os
import re

class ModbusDataset():
    """
    A class to organize and manage the CICModbus 2023 directories with simple hierarchical structure
    and metadata.
    """
    def __init__(self, _root_dir = "./ModbusDataset"):
        self.root_dir = _root_dir
        datasets_dir = self.find_csv_in_folder(root_dir,"output")
        benign_datasets_dir = self.find_csv_in_list(datasets_dir,"benign")
        attack_dataset_dir = self.find_csv_in_list(datasets_dir,"attack")
        ext_attack_dataset_dir = self.find_csv_in_list(attack_dataset_dir,"external")
        comp_ied_attack_dataset_dir = self.find_csv_in_list(attack_dataset_dir,"compromised-ied")
        comp_scada_attack_dataset_dir =self.find_csv_in_list(attack_dataset_dir,"compromised-scada")
        attack_logs_dir = self.find_csv_in_folder(root_dir,"attack logs")
        attack_logs_dir.extend(self.find_csv_in_folder(root_dir,"attacker logs"))
        ## Corrupted TimeStamp (more detail in ./ModbusDataset/Readme.md)
        attack_logs_dir.remove("./ModbusDataset/attack/compromised-scada/attack logs/03-21-2023/03-21-2023-1-original.csv")
        ext_attack_log_dir = self.find_csv_in_list(attack_logs_dir,"external")
        comp_ied_attack_log_dir = self.find_csv_in_list(attack_logs_dir,"compromised-ied")
        comp_scada_attack_log_dir =self.find_csv_in_list(attack_logs_dir,"compromised-scada")


        self.dataset ={
            "metadata":{
                "total_dataset_num":len(datasets_dir),"benign_dataset_num":len(benign_datasets_dir),"attack_dataset_num":{
                "total_num":len(attack_dataset_dir),
                "external_num":len(ext_attack_dataset_dir),
                "compromised-ied_num":len(comp_ied_attack_dataset_dir),
                "compromised-scada_num":len(comp_scada_attack_dataset_dir),
                },"attack_logs_num":{
                    "total_num":len(attack_logs_dir),
                    "external_num":(ext_attack_log_dir),
                    "compromised-ied_num":len(comp_ied_attack_log_dir),
                    "compromised-scada_num":len(comp_scada_attack_log_dir),
                }
            },
            "benign_dataset_dir":benign_datasets_dir,
            "attack_dataset_dir":{"total":attack_dataset_dir,
                                  "external":ext_attack_dataset_dir,
                                  "compromised-ied":comp_ied_attack_dataset_dir,
                                  "compromised-scada":comp_scada_attack_dataset_dir}
            ,
            "attack_log_dir":{"total":attack_logs_dir,
                            "external":ext_attack_log_dir,
                            "compromised-ied":comp_ied_attack_log_dir,
                            "compromised-scada":comp_scada_attack_log_dir}}
        
    def find_csv_in_folder(self,_start_path,_folder_name):
        csv_files = []
        for root, _, files in os.walk(_start_path):
            if _folder_name in root.split(os.sep) :
                csv_files.extend([os.path.join(root, f) for f in files if f.endswith('.csv')])
        return csv_files

    def find_csv_in_list(self,_datasets_dir,_folder_name):
        return  [ds for ds in _datasets_dir  if ds.find(_folder_name)!=(-1)]
    
    def summary_print(self):
        print("num of total fix-ordered dataset files ",self.dataset["metadata"]["total_dataset_num"])
        print("num of benign dataset files",self.dataset["metadata"]["benign_dataset_num"])
        print("num of attack dataset files",self.dataset["metadata"]["attack_dataset_num"]["total_num"])
        print("num of attack logs ",self.dataset["metadata"]["attack_logs_num"]["total_num"])
    
    def print_csv(datasets_dir_list,replace_dir):
        for i,dataset in enumerate(datasets_dir_list) :
            print(i+1,dataset.replace(replace_dir,""))

print("pandas current version ",pd.__version__)
print("numpy current version ",np.__version__)
root_dir = "./ModbusDataset"
modbus = ModbusDataset(root_dir)
modbus.summary_print()


pandas current version  2.2.0
numpy current version  1.26.3
num of total fix-ordered dataset files  170
num of benign dataset files 62
num of attack dataset files 108
num of attack logs  34


/tmp/ipykernel_89584/4155581274.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

print("list of benign csv dataset directories under ./ModbusDataset/benign/*")
ModbusDataset.print_csv(modbus.dataset["benign_dataset_dir"],"./ModbusDataset/benign/")
print("list of attack csv dataset directories under ./ModbusDataset/attack/*")
ModbusDataset.print_csv(modbus.dataset["attack_dataset_dir"]["total"],"./ModbusDataset/attack/")


list of benign csv dataset directories under ./ModbusDataset/benign/*
1 network-wide-pcap-capture/network-wide/output/network-wide-normal-18_fix_ord.pcap_Flow.csv
2 network-wide-pcap-capture/network-wide/output/network-wide-normal-22_fix_ord.pcap_Flow.csv
3 network-wide-pcap-capture/network-wide/output/network-wide-normal-30_fix_ord.pcap_Flow.csv
4 network-wide-pcap-capture/network-wide/output/network-wide-normal-19_fix_ord.pcap_Flow.csv
5 network-wide-pcap-capture/network-wide/output/network-wide-normal-32_fix_ord.pcap_Flow.csv
6 network-wide-pcap-capture/network-wide/output/network-wide-normal-25_fix_ord.pcap_Flow.csv
7 network-wide-pcap-capture/network-wide/output/network-wide-normal-15_fix_ord.pcap_Flow.csv
8 network-wide-pcap-capture/network-wide/output/network-wide-normal-31_fix_ord.pcap_Flow.csv
9 network-wide-pcap-capture/network-wide/output/network-wide-normal-24_fix_ord.pcap_Flow.csv
10 network-wide-pcap-capture/network-wide/output/network-wide-normal-14_fix_ord.pcap_Flow.csv

#### Labeling functions


In [53]:
# Basic preprocessing before getting started on labelling.
# Deletes rows with "Infinity" and NaNs, converts "Timestamp" to Pandas Datetime, and converts all necessary columns to numeric values
# Int_64 Columns (Attempted-Category) not considered.

print_index = False

def format_csv_for_labelling(df):
    df = df.replace('Infinity', np.nan)
    # Clean the Timestamp strings to always include microseconds (append .0 if missing)
    df['Timestamp'] = pd.to_datetime(
        df['Timestamp'].apply(
            lambda x: x if '.' in x.split()[-1] else f"{x}.0"  # Split into date/time and check time part
        ),
        format='%Y-%m-%d %H:%M:%S.%f'  # Parse with microseconds
    )
    for column in df.columns:
        if column not in ['Flow ID' , 'Timestamp', 'Src IP', 'Dst IP', 'Label','Attack','TransactionID','TargetIP']:
            df[column] = pd.to_numeric(df[column], errors='coerce')
    df.dropna()
    return df.dropna()

def read_csvs_from_path_and_reformat(path):
    df = pd.read_csv(path, encoding='cp1252')

    df = format_csv_for_labelling(df)
    print("labels after pre-processing:\n", df["Label"].value_counts())

    int32_columns = ["Src Port", "Dst Port", "Flow Duration", "Total Fwd Packet", "Total Bwd packets", "Total Length of Fwd Packet", "Total Length of Bwd Packet", "Fwd Packet Length Max",
        "Fwd Packet Length Min", "Bwd Packet Length Max", "Bwd Packet Length Min", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total",
        "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "Packet Length Min", "Packet Length Max", "FIN Flag Count", "SYN Flag Count", "RST Flag Count", "PSH Flag Count",
        "ACK Flag Count", "URG Flag Count", "CWR Flag Count", "ECE Flag Count", "Subflow Fwd Packets", "Subflow Fwd Bytes",
        "Subflow Bwd Packets", "Subflow Bwd Bytes", "FWD Init Win Bytes", "Bwd Init Win Bytes", "Fwd Act Data Pkts", "Fwd Seg Size Min", "Active Max",
        "Active Min", "Idle Max", "Idle Min"]

    int16_columns = ["Fwd Header Length", "Bwd Header Length", "ICMP Code", "ICMP Type"]

    for column in int32_columns:
        df[column] = df[column].astype('int32')

    for column in int16_columns:
        df[column] = df[column].astype('int16')

    return df


# This function is called when all labelling of malicious flows is completed. Anything that has not yet received a label
# so far is labelled as Benign.
# write dataframe to csv file in ready directory (create if not exist )
def label_rest_as_benign_and_write_csv(df, input_path):
    
    df["Label"]=df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN")
    print("label count after labelling:\r\n", df["Label"].value_counts())
    # Construct the ready directory path alongside output
    ready_dir = os.path.join(os.path.dirname(os.path.dirname(input_path)), 'ready')
    os.makedirs(ready_dir, exist_ok=True)
    filename = os.path.basename(input_path)
    name, ext = os.path.splitext(filename)  # Split the extension (name, ".csv")
    cleaned_name = name.replace("_fix_ord.pcap_Flow", "-labeled")
    # Construct full output path
    output_path = os.path.join(ready_dir, f"{cleaned_name}{ext}")  # Append extension
    # Adds line numbers in the first column if print_index is set to true
    if print_index:
        df.reset_index(inplace=True, drop=True)
        df.index += 1
        df.index.name = 'id'
        df.to_csv(output_path)
    else:
        df.to_csv(output_path, index=False)

def sort_df(_df):
    """    
    return sorted dataframe vertically with respect to TimeStamps
    """    
    new_df_sorted = _df.sort_values(by='Timestamp', ascending=True)
    new_df_sorted = new_df_sorted.reset_index(drop=True)
    return new_df_sorted

def merge_datasets(_dataset_dir):
    """
        create dataframes from the directories then return the merged_file  
    """
    # take two columns (attack logs ) or four columns (attacker logs)    
    merged_new_df = format_csv_for_labelling(pd.read_csv(_dataset_dir[0], encoding='cp1252'))
    for _dir in _dataset_dir :
        merged_new_df = pd.concat([format_csv_for_labelling(pd.read_csv(_dir, encoding='cp1252')),merged_new_df],join="inner")
    return merged_new_df
    

#### Label All csv files in benign folder as "BENIGN"

In [ ]:
# create new csv files with new labels in ready folder (run it once)
for _dir in modbus.dataset["benign_dataset_dir"]:
    print(_dir)
    # uncomment next line of code if the labeling of the benign dataset does not occured yet !
    # label_rest_as_benign_and_write_csv(read_csvs_from_path_and_reformat(_dir) ,_dir)
            

./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-18_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-22_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-30_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-19_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-32_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-25_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-15_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal-31_fix_ord.pcap_Flow.csv
./ModbusDataset/benign/network-wide-pcap-capture/network-wide/output/network-wide-normal

#### Attack Labeling depends on the log Attack , Timestamp columns



In the log only the completed attacks happened in pcaps. for example 
```md
2023-02-01 13:06:29.2, Brute force or specific coil. Address: 0
2023-02-01 13:06:29.201, Brute force or specific - Complete
```

🟢 It's okay but 
```md
2023-02-01 13:06:29.2, Brute force or specific coil. Address: 0
```
🔴 It didn't happen anymore. 

❗Unfortunately no exact logging happend in attack/attacker logs
Complete Timestamp of an attack might be before ending the flow/connection or after that !!!


In [ ]:
def clean_label(label_str):
    """Clean label by removing multiple unwanted substrings/characters"""
    # Remove "Complete" in any casing and commas
    label_str = re.sub(r'Complete|,|-', '', label_str, flags=re.IGNORECASE)
    # Remove any resulting double spaces or trailing hyphens
    label_str = re.sub(r'\s+', ' ', label_str).strip()
    label_str = re.sub(r'\s-\s*$', '', label_str)
    return label_str


def label_flows_from_log(_attack_dir,_attack_log_dirs,src_ip_list=None ,dst_ip_list=None):
    """
    label flows of the dataset csv file with respect to the logs overlaping timestamp 
    then change the _attack_dir "Label" Column.
    Timestamps are in NANOSECONDS (!) Unix time. Note that the CSV files are in the UTC timezone.
    """
    print("Attack directory ------------------------------",_attack_dir)
        
    # maximum allowed time difference (0.01 second = 10,000 microseconds)
    # between flow starts and attack start
    MAX_START_TIME_DIFF = np.timedelta64(10_000, 'us')
    # maximum allowed time difference (1 second = 1000,000 microseconds)
    # between flow end and attack ends 

    MAX_END_TIME_DIFF = np.timedelta64(1_000_000, 'us')

    attack_df= read_csvs_from_path_and_reformat(_attack_dir)
    attack_df['end_time'] = attack_df['Timestamp'] + pd.to_timedelta(attack_df['Flow Duration'], unit='us')
    attack_df=sort_df(attack_df)

    # sort attak_df beacuse of overlapped flow durations results in multiple labels 
    # Priority: Earlier flows in attack_df take precedence for labeling when multiple flows contain attack interval
    attack_log_df = sort_df(merge_datasets(_attack_log_dirs))
    
    
    #  Find rows in attack_log_df where 'Attack' contains 'Complete'
    complete_mask = attack_log_df['Attack'].str.contains('Complete', na=False)
    # Find valid attack pairs and convert to numpy datetime64 upfront
    pairs = []
    for i in attack_log_df.index[complete_mask]:
            if i > 0:
                start = np.datetime64(attack_log_df.at[i-1, 'Timestamp'],'us')
                end = np.datetime64(attack_log_df.at[i, 'Timestamp'],'us')
                label = clean_label(attack_log_df.at[i, 'Attack'])
                pairs.append((start, end, label))

    # Convert to numpy arrays for vectorized operations
    starts = attack_df['Timestamp'].values.astype('datetime64[us]')
    ends = attack_df['end_time'].values.astype('datetime64[us]')
    labels = attack_df['Label'].values
    orig_indices = attack_df.index.values
    
    # Precompute mask for unlabeled flows
    unlabeled_mask = (labels == "NeedManualLabel")

    # Process each attack pair
    for start_p, end_p, new_label in pairs:
        # Find flows that:
        # 1. Contain the entire attack interval (start_p to end_p)
        # 2. Start within 0.1 seconds of attack start
        # 3. Are still unlabeled
        # contains_interval = (starts <= start_p) & (np.abs(start_p-starts) <= MAX_START_TIME_DIFF) & (ends >= end_p) & unlabeled_mask
        contains_interval = (starts <= start_p) & (np.abs(start_p-starts) <= MAX_START_TIME_DIFF) & (np.abs(ends-end_p) <= MAX_END_TIME_DIFF) & unlabeled_mask

        matching_indices = np.where(contains_interval)[0]
        
        if len(matching_indices) > 0:
            # Get first matching flow by original row order
            first_match_idx = matching_indices[0]
            orig_idx = orig_indices[first_match_idx]
            
            # Update label in original dataframe
            attack_df.loc[orig_idx, 'Label'] = new_label
            
            # Update unlabeled mask to mark this flow as labeled
            unlabeled_mask[first_match_idx] = False
    label_rest_as_benign_and_write_csv(attack_df,_attack_dir)




##### External Attacks 
- Attacks from an unknown IP address (185.175.0.7) targeting 185.175.0.4.

In [ ]:
for ext_att_dir in modbus.dataset["attack_dataset_dir"]["external"] :
    # uncomment next line of code if the labeling of the external attack dataset does not occured yet  !
    # label_flows_from_log(ext_att_dir,modbus.dataset["attack_log_dir"]["external"],src_ip_list=["185.175.0.7"])


Attack directory ------------------------------ ./ModbusDataset/attack/external/network-wide/output/network-wide-normal-1_fix_ord.pcap_Flow.csv
labels after pre-processing:
 Label
NeedManualLabel    68931
Name: count, dtype: int64
label count after labelling:
 Label
Brute force or specific    54112
BENIGN                     14819
Name: count, dtype: int64
Attack directory ------------------------------ ./ModbusDataset/attack/external/network-wide/output/network-wide-normal-0_fix_ord.pcap_Flow.csv
labels after pre-processing:
 Label
NeedManualLabel    152398
Name: count, dtype: int64
label count after labelling:
 Label
BENIGN                     141328
Brute force or specific     11035
Recon.                         31
Query flooding.                 1
Length manipulation             1
Payload injection               1
Stacked Modbus Frames           1
Name: count, dtype: int64
Attack directory ------------------------------ ./ModbusDataset/attack/external/ied4c/ied4c-network-capture/o

##### Compromised SCADA Attacks 
- Attacks from 185.175.0.3 targeting 185.175.0.4 and 185.175.0.8.
- Days 12 to 15 missed in attack logs.

##### Compromised IED Attacks
- Attacks from 185.175.0.5 targeting 185.175.0.2.